In [20]:
from pathlib import Path
import scipy.io as sio
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [21]:
DATA_PATH_C = Path("../data/russo-2018/Cousteau_tt.mat")
DATA_PATH_D = Path("../data/russo-2018/Drake_tt.mat")
data_c = sio.loadmat(DATA_PATH_C)['Pc'][0,0]

In [22]:
# Separating the trial-averaged data into different rows of a Pandas dataframe for each task condition.
# (The 20 different conditions were concatenated in the original dataset)

# Pre-processed trial-average activity in the 116 M1 neurons.
# Data was "soft normalised" using the formula response := response / (range(response) + 5). This reduces the normalised activity of weakly active neurons more than strongly active neurons.
processed_neuron_activity = data_c["xA"]

# Time after trial start for each neural activity sample
activity_time_after_start = data_c["mask"]["time"][0][0][:,0]

# Labels for different trial types:
# distance: number of cycles of the pedal required
# forward: whether the pedal had to be moved in the forward or backward direction
# top_start: thether the pedal position started at the bottom or top of its range of positions
distance = data_c["mask"]["dist"][0][0][:,0]
forward = data_c["mask"]["dir"][0][0][:,0] == 1
top_start = data_c["mask"]["pos"][0][0][:,0] == 0.5

values = []
data = processed_neuron_activity
timestamp = activity_time_after_start
start_index = 0
end_index = 0
for i in range(len(timestamp)):
    if i < len(timestamp) - 1:
        if timestamp[i] <= timestamp[i+1]:
            end_index += 1
            continue
    values.append([data[start_index:end_index+1], timestamp[start_index:end_index+1], forward[start_index], top_start[start_index], distance[start_index]])
    end_index += 1
    start_index = end_index

separated_trials = pd.DataFrame(values, columns = ["data", "time", "forward", "top_start", "cycles"])

In [23]:
# Soft normalised neural activity for two M1 neurons
fig0 = plt.figure()
ax0 = fig0.add_subplot(1,1,1)
ax0.plot(separated_trials["time"][3], separated_trials["data"][3][:,10], alpha=.6)
ax0.plot(separated_trials["time"][3], separated_trials["data"][3][:,5], alpha=.6)
ax0.set(xlabel='Time After Trial Start (s)', ylabel='Neuron Activity (Processed)')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
%matplotlib widget

# Data from monkey C showing similar results after PCA to those in figure 4.
# Exact replication of that figure was not possible, as it is unclear which data was used to produce it.
chosen_data = separated_trials["data"][3][(separated_trials["time"][3] < 2) & (separated_trials["time"][3] > 1)]

# Performing the principle components analysis on the selected data
pca = PCA(n_components = 3)
pca.fit(chosen_data)

# Map the 116-dimensional trial-averaged data from one condition into the 3-dimensional PCA coordinate system
X = pca.transform(separated_trials["data"][3])

# Plot the trajectory in the PCA coordinate system 
fig = plt.figure()
ax = fig.add_subplot(2,1,1, projection='3d')

ax.plot(X[:, 0], X[:, 1], X[:, 2], alpha=.9)
ax.set(xlabel='PC1', ylabel='PC2', zlabel="PC3")

ax2 = fig.add_subplot(2,1,2)
ax2.plot(X[:, 0], X[:, 1], )
ax2.set(xlabel='PC1', ylabel='PC2', aspect="equal")

plt.show()

sum(pca.explained_variance_ratio_[1:3])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.3539967715903748